[Krish Naik Video Link](https://www.youtube.com/watch?v=f-AXdiCyiT8&list=PLZoTAELRMXVNOWh1SDXt5NFujQMOt-CWy&index=3)

In [1]:
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-langchain

We need bistandbytes for quantization, as most of the models will be in 16bits. We will quantise them to 4 bits

In [2]:
!pip install -q transformers einops accelerate langchain bitsandbytes llama_index pypdf

In [3]:
## Install for Embeddings
!pip install sentence_transformers

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

### RAG with LlamaIndex and Llama-2

In [5]:
import os, glob
path = os.path.join(os.getcwd(), "drive","MyDrive","LLM","data")
#glob.glob(path+'/*')

In [6]:
# Service Context enables combine Llama2 model with prompt template
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [7]:
documents = SimpleDirectoryReader(path).load_data()

In [8]:
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

# Default Format supported by Llama2

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|")

In [9]:
import torch

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

llm = HuggingFaceLLM(
    context_window = 4096,
    max_new_tokens = 256,
    generate_kwargs={"temperature":0.0, "do_sample":False},
    system_prompt = system_prompt,
    query_wrapper_prompt = query_wrapper_prompt,
    tokenizer_name = "meta-llama/Llama-2-7b-chat-hf",
    model_name = "meta-llama/Llama-2-7b-chat-hf",
    device_map = "auto",
    model_kwargs = {"torch_dtype":torch.float16, "load_in_8bit":True}
)

embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

At this points we have the following:
- Embedding Model
- LLM Model
- Documents

We will now combine all of them with the help of __service context__

In [10]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    chunk_size=1024
)

<ipython-input-10-5a24b1a53139>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [13]:
index = VectorStoreIndex.from_documents(documents, service_context = service_context)
query_engine = index.as_query_engine()

In [16]:
response = query_engine.query("How is the overall market outlook")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


> The overall market outlook for the top 20 5G markets is positive, with continued revenue growth expected in leading 5G markets. According to the Ericsson Mobility Report, service providers are expected to continue to drive a shift to higher mobile tariffs while adding value for consumers. The report projects that 5G subscriptions will grow from 13 to 70 million between 2022 and 2028, accounting for 86% of total subscriptions at the end of the period. Additionally, the report states that 4G is expected to decline by 29% annually as subscribers increasingly move to 5G, and service providers will continue to explore enterprise opportunities.

In Central and Eastern Europe, technology adoption and subscription uptake are typically slower than in Western Europe due to slower spectrum allocation processes and consumers being reluctant to upgrade to more expensive subscriptions. However, the migration from 2G/3G to 4G continues to look strong up to 2024, and from 2025, 5G is expected to be 